## EDA and Classify using Embeddings

In [5]:
import pandas as pd
from utils import drug_data_tools

In [3]:
drug_classes = pd.read_csv("outputs/drug-classes.csv", index_col=0)#, index_col="Drug Name")
drug_classes

,Drug Class
Drug Name,
ella,Progesterone receptor modulators
pancreatin,Digestive enzymes
boroleum,Topical rubefacient
benicar,Angiotensin receptor blockers
duac,Topical acne agents
...,...
acne-10-gel-topical,Topical acne agents
lorbrena,Multikinase inhibitors
halcion,Benzodiazepines


In [8]:
selected_molecules = drug_data_tools.get_selected_molecules()
selected_molecules[:10]

['amoxicillin',
 'metoprolol',
 'phenoxymethylpenicillin',
 'azithromycin',
 'celiprolol',
 'simvastatin',
 'vancomycin',
 'ibuprofen',
 'tadalafil',
 'timolol']

In [9]:
# Drugs that are both in Drugs.com and Selected Molecules
len(set(drug_classes.index).intersection(selected_molecules))

115

In [10]:
# How many drugs.com drugs are also in Selected Molecules
intersecting_drugs = drug_classes[drug_classes.index.isin(set(drug_classes.index).intersection(selected_molecules))]
# intersecting_drugs

In [ ]:
intersecting_drugs['Drug Class'].value_counts()

Drug Class
Multikinase inhibitors                                 18
Statins                                                 7
Protease inhibitors                                     6
Angiotensin receptor blockers                           6
Local injectable anesthetics                            5
Cardioselective beta blockers                           5
Antibiotics / antineoplastics                           4
Purine nucleosides                                      4
BCR-ABL tyrosine kinase inhibitors                      4
EGFR inhibitors                                         4
Miscellaneous antivirals                                4
Macrolides                                              3
Heparins                                                3
Ophthalmic glaucoma agents                              3
Non-cardioselective beta blockers                       3
Integrase strand transfer inhibitor                     3
Nonsteroidal anti-inflammatory drugs                    3
Nuc

In [ ]:
# Nr unique drug classes
len(set(intersecting_drugs['Drug Class']))

37

In [ ]:
# TODO Why are some drugs missing? 115 vs 103
intersecting_drugs_w_embd_df = df_embeddings.loc[df_embeddings.index.intersection(set(intersecting_drugs.index))]
intersecting_drugs_w_embd_df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
Drug Name,,,,,,,,,,,,,,,,,,,,,
natalizumab,2.828465,-0.323880,0.461534,1.198783,0.935547,0.000823,0.155675,-0.988050,-1.114025,-0.150321,...,0.569156,0.244178,1.626112,0.391793,-3.204831,2.612899,-0.117829,-0.720186,-1.816628,0.691926
candesartan,0.846399,-1.726462,0.906651,-1.306898,0.193972,0.240104,0.303152,-1.743013,-1.192302,-1.672895,...,0.621502,-0.167140,0.252433,0.113513,-1.079827,-0.638547,1.297506,0.620969,0.505722,-1.139408
pemigatinib,1.657495,-1.523324,0.316984,0.202013,-0.703365,0.021658,-1.032712,-1.622851,0.565182,-2.019764,...,0.295195,-0.309789,-1.277183,0.816564,-0.307367,1.484333,0.583064,0.173681,-1.643754,0.549972
telmisartan,1.801593,-2.129047,0.749666,-0.453170,0.827487,-0.062932,0.020983,-0.774632,-0.643007,-2.144073,...,0.582292,0.434012,0.108160,0.781245,-1.225841,0.251356,0.407616,0.496148,0.615039,-1.151465
fluvastatin,1.884299,-1.708703,1.452643,-3.322976,0.765282,0.389487,1.642352,-0.425322,-0.315836,-1.930144,...,-0.278033,-0.646324,0.640017,0.383189,-0.533273,-2.756398,1.953617,-1.458458,-0.056362,0.846975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sofosbuvir,0.019505,0.669670,-2.041065,-2.784205,0.027118,0.313708,1.415276,-2.556959,-0.927886,0.875131,...,-0.203469,1.499896,0.643817,3.036791,0.464148,2.561702,0.132109,-0.862448,-2.124450,0.070136
darunavir,0.206294,-0.861255,-1.311981,-0.856387,2.775300,0.372177,2.384961,-0.891477,-0.670463,-0.338599,...,-0.455320,2.972468,1.895652,1.195464,-0.460290,2.032736,-2.124213,1.353184,-2.773972,-1.118577
pentostatin,1.015859,-1.931853,0.461308,0.744791,0.934850,0.227804,0.968633,-1.305325,0.316753,-1.942161,...,0.633655,0.172523,0.602261,-0.246623,-1.375640,-0.609349,1.551445,-0.746172,0.167813,0.155121


In [ ]:
intersecting_drugs.loc[intersecting_drugs_w_embd_df.index]

,Drug Class
Drug Name,
natalizumab,Selective immunosuppressants
candesartan,Angiotensin receptor blockers
pemigatinib,Multikinase inhibitors
telmisartan,Angiotensin receptor blockers
fluvastatin,Statins
...,...
sofosbuvir,Miscellaneous antivirals
darunavir,Protease inhibitors
pentostatin,Antibiotics / antineoplastics


In [ ]:
intersecting_drugs_w_embd_df = pd.concat([intersecting_drugs_w_embd_df, intersecting_drugs.loc[intersecting_drugs_w_embd_df.index]['Drug Class']], axis=1)
intersecting_drugs_w_embd_df

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,Drug Class
Drug Name,,,,,,,,,,,,,,,,,,,,,
natalizumab,2.828465,-0.323880,0.461534,1.198783,0.935547,0.000823,0.155675,-0.988050,-1.114025,-0.150321,...,0.244178,1.626112,0.391793,-3.204831,2.612899,-0.117829,-0.720186,-1.816628,0.691926,Selective immunosuppressants
candesartan,0.846399,-1.726462,0.906651,-1.306898,0.193972,0.240104,0.303152,-1.743013,-1.192302,-1.672895,...,-0.167140,0.252433,0.113513,-1.079827,-0.638547,1.297506,0.620969,0.505722,-1.139408,Angiotensin receptor blockers
pemigatinib,1.657495,-1.523324,0.316984,0.202013,-0.703365,0.021658,-1.032712,-1.622851,0.565182,-2.019764,...,-0.309789,-1.277183,0.816564,-0.307367,1.484333,0.583064,0.173681,-1.643754,0.549972,Multikinase inhibitors
telmisartan,1.801593,-2.129047,0.749666,-0.453170,0.827487,-0.062932,0.020983,-0.774632,-0.643007,-2.144073,...,0.434012,0.108160,0.781245,-1.225841,0.251356,0.407616,0.496148,0.615039,-1.151465,Angiotensin receptor blockers
fluvastatin,1.884299,-1.708703,1.452643,-3.322976,0.765282,0.389487,1.642352,-0.425322,-0.315836,-1.930144,...,-0.646324,0.640017,0.383189,-0.533273,-2.756398,1.953617,-1.458458,-0.056362,0.846975,Statins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sofosbuvir,0.019505,0.669670,-2.041065,-2.784205,0.027118,0.313708,1.415276,-2.556959,-0.927886,0.875131,...,1.499896,0.643817,3.036791,0.464148,2.561702,0.132109,-0.862448,-2.124450,0.070136,Miscellaneous antivirals
darunavir,0.206294,-0.861255,-1.311981,-0.856387,2.775300,0.372177,2.384961,-0.891477,-0.670463,-0.338599,...,2.972468,1.895652,1.195464,-0.460290,2.032736,-2.124213,1.353184,-2.773972,-1.118577,Protease inhibitors
pentostatin,1.015859,-1.931853,0.461308,0.744791,0.934850,0.227804,0.968633,-1.305325,0.316753,-1.942161,...,0.172523,0.602261,-0.246623,-1.375640,-0.609349,1.551445,-0.746172,0.167813,0.155121,Antibiotics / antineoplastics
